In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [2]:
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPool1D, Flatten, Dropout,LSTM

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Raw_data = '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/only data/raw_data.npz'

Theta_data = '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/time frequency domain/RAW/Alpha_merged_features.npz'



# RAW = np.load(Raw_data, allow_pickle=True)  # Allow loading pickled object arrays
# X = RAW['X'].astype('float64')
# Y = RAW['Y'].astype('float64')
# group = RAW['Group'].astype('float64')

This is a good performing model

# CNN-LSTM

In [7]:
%%capture
!pip install tensorflow_addons

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, cohen_kappa_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization, LSTM, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.backend import clear_session
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
import tensorflow_addons as tfa

class FederatedData:
    def __init__(self, data_path, num_clients, scaler_type='MinMax', test_size=0.2):
        self.data_path = data_path
        self.num_clients = num_clients
        self.scaler_type = scaler_type
        self.test_size = test_size
        self.partitions = []  # Initialize partitions attribute
        self.load_data()
        self.scale_data()
        self.create_partitions()

    def load_data(self):
        try:
            data = np.load(self.data_path, allow_pickle=True)
            self.X = np.moveaxis(data['X'], 1, 2)  # Move axis here
            self.Y = data['Y']
            self.groups = data['Group']  # Assuming 'Group' contains group identifiers
        except KeyError as e:
            raise ValueError(f"Missing expected data field: {e}")
        except FileNotFoundError as e:
            raise ValueError(f"Data file not found: {e}")

    def scale_data(self):
        # Reshape data to 2D array for scaling
        X_reshaped = self.X.reshape(-1, self.X.shape[-1])

        # Select scaler based on input
        if self.scaler_type == 'Standard':
            scaler = StandardScaler()
        elif self.scaler_type == 'MinMax':
            scaler = MinMaxScaler(feature_range=(0, 1))
        else:
            raise ValueError("Unsupported scaler type. Choose either 'Standard' or 'MinMax'.")

        # Fit and transform the data
        scaled_data_reshaped = scaler.fit_transform(X_reshaped)

        # Reshape back to original shape
        self.X = scaled_data_reshaped.reshape(self.X.shape)

    def create_partitions(self):
        gss = GroupShuffleSplit(n_splits=self.num_clients, test_size=self.test_size, random_state=42)
        for train_index, test_index in gss.split(self.X, self.Y, self.groups):
            X_train, X_test = self.X[train_index], self.X[test_index]
            Y_train, Y_test = self.Y[train_index], self.Y[test_index]
            self.partitions.append((X_train, Y_train, X_test, Y_test))
            print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

    def get_training_and_validation_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        partition_X_train, partition_Y_train, partition_X_test, partition_Y_test = self.partitions[client_idx]
        X_train, X_val, Y_train, Y_val = train_test_split(partition_X_train, partition_Y_train, test_size=0.2, random_state=42)
        return X_train, X_val, Y_train, Y_val, partition_X_test, partition_Y_test

    def get_testing_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        _, _, partition_X_test, partition_Y_test = self.partitions[client_idx]
        return partition_X_test, partition_Y_test

def build_sequential_model(input_shape):
    clear_session()
    model = Sequential()

    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same", input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(512, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(filters=1024, kernel_size=3, strides=1, activation='relu'))
    model.add(Dropout(0.5))
    model.add(LSTM(256, return_sequences=True))
    model.add(LSTM(256, return_sequences=True))

    model.add(Flatten())
    model.add(Dense(512, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(256, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(32, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.001)))

    # opt = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    opt = tfa.optimizers.RectifiedAdam(learning_rate=0.00009)
    opt = tfa.optimizers.SWA(opt)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = recall

    return accuracy, precision, recall, f1, sensitivity, specificity, kappa

def federated_learning(data_path):
    federated_data = FederatedData(data_path, num_clients=3, scaler_type='MinMax')
    federated_data.create_partitions()

    # Get the input shape from the data
    input_shape = federated_data.X.shape[1:]
    global_model = build_sequential_model(input_shape)

    num_clients = 3
    local_epochs = 5
    global_optimizer = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    # Initialize m and v for Adam optimizer
    m = [np.zeros_like(w) for w in global_model.get_weights()]
    v = [np.zeros_like(w) for w in global_model.get_weights()]
    beta1 = 0.9
    beta2 = 0.999
    epsilon = 1e-7
    t = 0

    client_data = []
    for client_idx in range(num_clients):
        x_train, x_val, y_train, y_val, x_test, y_test = federated_data.get_training_and_validation_data(client_idx)
        client_data.append((x_train, x_val, y_train, y_val, x_test, y_test))

    metrics_list = []

    for epoch in range(local_epochs):
        client_models = []

        for client in range(num_clients):
            x_train, x_val, y_train, y_val, x_test, y_test = client_data[client]
            client_model = build_sequential_model(input_shape)
            client_model.set_weights(global_model.get_weights())  # Initialize with global weights
            opt = Adam(learning_rate=0.000009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
            client_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

            # Set up callbacks
            run_name = f"epoch_{epoch}_client_{client}"
            checkpoint_filepath = f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time+Frequency/Alpha/best_model_{run_name}.h5'
            checkpoint_callback = ModelCheckpoint(
                checkpoint_filepath,
                monitor='val_accuracy',
                save_best_only=True,
                mode='max'
            )
            csv_logger = CSVLogger(f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time+Frequency/Alpha/training_log_{run_name}.csv', append=True, separator=';')

            history = client_model.fit(x_train, y_train, epochs=100, batch_size=128, validation_data=(x_val, y_val), callbacks=[checkpoint_callback, csv_logger])
            client_models.append(client_model)

            print(history.history)

            # Load the best model before evaluation
            best_model = load_model(checkpoint_filepath)

            # Evaluate client model
            y_pred = (best_model.predict(x_test) > 0.5).astype("int32")
            accuracy, precision, recall, f1, sensitivity, specificity, kappa = compute_metrics(y_test, y_pred)
            metrics_list.append({
                'Client': client,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1': f1,
                'Sensitivity': sensitivity,
                'Specificity': specificity,
                'Kappa': kappa
            })

        global_weights = global_model.get_weights()
        layer_start_idx = 0
        for layer in global_model.layers:
            layer_weights = []
            layer_biases = []
            num_params = len(layer.get_weights())
            for i in range(num_clients):
                layer_params = client_models[i].get_weights()[layer_start_idx:layer_start_idx + num_params]
                if num_params > 0:
                    layer_weights.append(layer_params[0])
                if num_params > 1:
                    layer_biases.append(layer_params[1])

            if len(layer_weights) > 0:
                averaged_layer_weights = np.mean(layer_weights, axis=0)
                if averaged_layer_weights.shape == global_weights[layer_start_idx].shape:
                    global_weights[layer_start_idx] = averaged_layer_weights
                else:
                    print(f"Warning: Shape mismatch for weights at layer {layer.name}, expected {global_weights[layer_start_idx].shape} but got {averaged_layer_weights.shape}")
            if len(layer_biases) > 0:
                averaged_layer_biases = np.mean(layer_biases, axis=0)
                if averaged_layer_biases.shape == global_weights[layer_start_idx + 1].shape:
                    global_weights[layer_start_idx + 1] = averaged_layer_biases
                else:
                    print(f"Warning: Shape mismatch for biases at layer {layer.name}, expected {global_weights[layer_start_idx + 1].shape} but got {averaged_layer_biases.shape}")

            layer_start_idx += num_params

        # Apply FedOpt (Adam) update to global weights
        t += 1
        for i in range(len(global_weights)):
            g_t = global_weights[i] - global_model.get_weights()[i]  # Gradient
            m[i] = beta1 * m[i] + (1 - beta1) * g_t
            v[i] = beta2 * v[i] + (1 - beta2) * (g_t ** 2)
            m_hat = m[i] / (1 - beta1 ** t)
            v_hat = v[i] / (1 - beta2 ** t)
            global_weights[i] -= global_optimizer.learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)

        global_model.set_weights(global_weights)

    metrics_df = pd.DataFrame(metrics_list)
    return global_model, metrics_df

# Example usage
# data_path = 'path/to/your/data.npz'
# global_model, metrics_df = federated_learning(data_path)
# print(metrics_df)


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
global_model, metrics_df = federated_learning(Theta_data)


Train shape: (4640, 37, 29), Test shape: (1194, 37, 29)
Train shape: (4418, 37, 29), Test shape: (1416, 37, 29)
Train shape: (4838, 37, 29), Test shape: (996, 37, 29)
Train shape: (4640, 37, 29), Test shape: (1194, 37, 29)
Train shape: (4418, 37, 29), Test shape: (1416, 37, 29)
Train shape: (4838, 37, 29), Test shape: (996, 37, 29)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 37, 256)           22528     
                                                                 
 batch_normalization (Batch  (None, 37, 256)           1024      
 Normalization)                                                  
                                                                 
 max_pooling1d (MaxPooling1  (None, 19, 256)           0         
 D)                                                              
                                                    

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


29/29 [==============================] - 27s 192ms/step - loss: 1.7001 - accuracy: 0.4884 - val_loss: 1.6948 - val_accuracy: 0.4849
Epoch 2/100
29/29 [==============================] - 1s 27ms/step - loss: 1.6899 - accuracy: 0.5040 - val_loss: 1.6847 - val_accuracy: 0.4849
Epoch 3/100
29/29 [==============================] - 1s 30ms/step - loss: 1.6798 - accuracy: 0.5059 - val_loss: 1.6747 - val_accuracy: 0.4849
Epoch 4/100
29/29 [==============================] - 1s 21ms/step - loss: 1.6699 - accuracy: 0.5040 - val_loss: 1.6648 - val_accuracy: 0.4849
Epoch 5/100
29/29 [==============================] - 0s 15ms/step - loss: 1.6599 - accuracy: 0.5046 - val_loss: 1.6549 - val_accuracy: 0.4849
Epoch 6/100
29/29 [==============================] - 0s 17ms/step - loss: 1.6502 - accuracy: 0.5030 - val_loss: 1.6452 - val_accuracy: 0.4849
Epoch 7/100
29/29 [==============================] - 0s 16ms/step - loss: 1.6405 - accuracy: 0.5046 - val_loss: 1.6356 - val_accuracy: 0.4849
Epoch 8/100
29/2

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 8s 88ms/step - loss: 1.7003 - accuracy: 0.4909 - val_loss: 1.6951 - val_accuracy: 0.4955
Epoch 2/100
28/28 [==============================] - 1s 21ms/step - loss: 1.6905 - accuracy: 0.5108 - val_loss: 1.6854 - val_accuracy: 0.4955
Epoch 3/100
28/28 [==============================] - 1s 21ms/step - loss: 1.6808 - accuracy: 0.4833 - val_loss: 1.6757 - val_accuracy: 0.4955
Epoch 4/100
28/28 [==============================] - 1s 21ms/step - loss: 1.6711 - accuracy: 0.4924 - val_loss: 1.6661 - val_accuracy: 0.4955
Epoch 5/100
28/28 [==============================] - 1s 21ms/step - loss: 1.6615 - accuracy: 0.5008 - val_loss: 1.6566 - val_accuracy: 0.4955
Epoch 6/100
28/28 [==============================] - 1s 20ms/step - loss: 1.6521 - accuracy: 0.5003 - val_loss: 1.6472 - val_accuracy: 0.4955
Epoch 7/100
28/28 [==============================] - 1s 20ms/step - loss: 1.6427 - accuracy: 0.5209 - val_loss: 1.6379 - val_accuracy: 0.4955
Epoch 8/100
28/28 

...

29/29 [==============================] - 7s 57ms/step - loss: 0.4731 - accuracy: 0.8798 - val_loss: 0.8828 - val_accuracy: 0.4849
Epoch 2/100
29/29 [==============================] - 0s 16ms/step - loss: 0.3851 - accuracy: 0.9154 - val_loss: 0.8847 - val_accuracy: 0.4849
Epoch 3/100
29/29 [==============================] - 0s 16ms/step - loss: 0.3762 - accuracy: 0.9178 - val_loss: 0.8853 - val_accuracy: 0.4849
Epoch 4/100
29/29 [==============================] - 0s 16ms/step - loss: 0.3633 - accuracy: 0.9219 - val_loss: 0.8875 - val_accuracy: 0.4849
Epoch 5/100
29/29 [==============================] - 0s 15ms/step - loss: 0.3963 - accuracy: 0.9089 - val_loss: 0.8864 - val_accuracy: 0.4849
Epoch 6/100
29/29 [==============================] - 1s 23ms/step - loss: 0.4231 - accuracy: 0.8941 - val_loss: 0.8804 - val_accuracy: 0.4860
Epoch 7/100
29/29 [==============================] - 0s 17ms/step - loss: 0.3791 - accuracy: 0.9151 - val_loss: 0.8840 - val_accuracy: 0.4860
Epoch 8/100
29/29 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 8s 80ms/step - loss: 0.4941 - accuracy: 0.8659 - val_loss: 0.8780 - val_accuracy: 0.4955
Epoch 2/100
28/28 [==============================] - 1s 22ms/step - loss: 0.4041 - accuracy: 0.9055 - val_loss: 0.8778 - val_accuracy: 0.4955
Epoch 3/100
28/28 [==============================] - 1s 26ms/step - loss: 0.3668 - accuracy: 0.9256 - val_loss: 0.8780 - val_accuracy: 0.4966
Epoch 4/100
28/28 [==============================] - 1s 24ms/step - loss: 0.3690 - accuracy: 0.9211 - val_loss: 0.8797 - val_accuracy: 0.4977
Epoch 5/100
28/28 [==============================] - 0s 18ms/step - loss: 0.3669 - accuracy: 0.9208 - val_loss: 0.8812 - val_accuracy: 0.4977
Epoch 6/100
28/28 [==============================] - 1s 24ms/step - loss: 0.3621 - accuracy: 0.9222 - val_loss: 0.8801 - val_accuracy: 0.4989
Epoch 7/100
28/28 [==============================] - 0s 18ms/step - loss: 0.3540 - accuracy: 0.9247 - val_loss: 0.8787 - val_accuracy: 0.4989
Epoch 8/100
28/28 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


31/31 [==============================] - 8s 65ms/step - loss: 0.5119 - accuracy: 0.8633 - val_loss: 0.8838 - val_accuracy: 0.4855
Epoch 2/100
31/31 [==============================] - 1s 22ms/step - loss: 0.3998 - accuracy: 0.9096 - val_loss: 0.8844 - val_accuracy: 0.4855
Epoch 3/100
31/31 [==============================] - 1s 30ms/step - loss: 0.3760 - accuracy: 0.9194 - val_loss: 0.8858 - val_accuracy: 0.4876
Epoch 4/100
31/31 [==============================] - 1s 21ms/step - loss: 0.3761 - accuracy: 0.9225 - val_loss: 0.8866 - val_accuracy: 0.4876
Epoch 5/100
31/31 [==============================] - 1s 20ms/step - loss: 0.3724 - accuracy: 0.9181 - val_loss: 0.8904 - val_accuracy: 0.4866
Epoch 6/100
31/31 [==============================] - 1s 21ms/step - loss: 0.3813 - accuracy: 0.9093 - val_loss: 0.8877 - val_accuracy: 0.4866
Epoch 7/100
31/31 [==============================] - 1s 16ms/step - loss: 0.3919 - accuracy: 0.9065 - val_loss: 0.8858 - val_accuracy: 0.4876
Epoch 8/100
31/31 

In [ ]:

metrics_df.round(3)

,Client,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,Kappa
0,0,0.532,0.544,0.390,0.455,0.390,0.673,0.064
1,1,0.537,0.546,0.442,0.489,0.442,0.633,0.075
2,2,0.561,0.555,0.616,0.584,0.616,0.506,0.122
3,0,0.580,0.585,0.556,0.570,0.556,0.605,0.161
4,1,0.602,0.601,0.610,0.605,0.610,0.595,0.205
5,2,0.624,0.618,0.651,0.634,0.651,0.598,0.249
6,0,0.642,0.641,0.645,0.643,0.645,0.638,0.283
7,1,0.665,0.645,0.732,0.686,0.732,0.597,0.329
8,2,0.688,0.680,0.709,0.694,0.709,0.667,0.376
9,0,0.708,0.718,0.683,0.700,0.683,0.732,0.415


In [ ]:
metrics_df.round(4).to_csv('/content/drive/MyDrive/EEG Signal /results/FLresults/TF Domain/Alpha_CNN_LSTM.csv', index = False)

# CNN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, cohen_kappa_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization, LSTM, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.backend import clear_session
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
import tensorflow_addons as tfa

class FederatedData:
    def __init__(self, data_path, num_clients, scaler_type='MinMax', test_size=0.2):
        self.data_path = data_path
        self.num_clients = num_clients
        self.scaler_type = scaler_type
        self.test_size = test_size
        self.partitions = []  # Initialize partitions attribute
        self.load_data()
        self.scale_data()
        self.create_partitions()

    def load_data(self):
        try:
            data = np.load(self.data_path, allow_pickle=True)
            self.X = np.moveaxis(data['X'], 1, 2)  # Move axis here
            self.Y = data['Y']
            self.groups = data['Group']  # Assuming 'Group' contains group identifiers
        except KeyError as e:
            raise ValueError(f"Missing expected data field: {e}")
        except FileNotFoundError as e:
            raise ValueError(f"Data file not found: {e}")

    def scale_data(self):
        # Reshape data to 2D array for scaling
        X_reshaped = self.X.reshape(-1, self.X.shape[-1])

        # Select scaler based on input
        if self.scaler_type == 'Standard':
            scaler = StandardScaler()
        elif self.scaler_type == 'MinMax':
            scaler = MinMaxScaler(feature_range=(0, 1))
        else:
            raise ValueError("Unsupported scaler type. Choose either 'Standard' or 'MinMax'.")

        # Fit and transform the data
        scaled_data_reshaped = scaler.fit_transform(X_reshaped)

        # Reshape back to original shape
        self.X = scaled_data_reshaped.reshape(self.X.shape)

    def create_partitions(self):
        gss = GroupShuffleSplit(n_splits=self.num_clients, test_size=self.test_size, random_state=42)
        for train_index, test_index in gss.split(self.X, self.Y, self.groups):
            X_train, X_test = self.X[train_index], self.X[test_index]
            Y_train, Y_test = self.Y[train_index], self.Y[test_index]
            self.partitions.append((X_train, Y_train, X_test, Y_test))
            print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

    def get_training_and_validation_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        partition_X_train, partition_Y_train, partition_X_test, partition_Y_test = self.partitions[client_idx]
        X_train, X_val, Y_train, Y_val = train_test_split(partition_X_train, partition_Y_train, test_size=0.2, random_state=42)
        return X_train, X_val, Y_train, Y_val, partition_X_test, partition_Y_test

    def get_testing_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        _, _, partition_X_test, partition_Y_test = self.partitions[client_idx]
        return partition_X_test, partition_Y_test

def build_sequential_model(input_shape):
    clear_session()
    model = Sequential()

    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same", input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(512, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(filters=1024, kernel_size=3, strides=1, activation='relu'))
    # model.add(Dropout(0.5))
    # model.add(LSTM(256, return_sequences=True))
    # model.add(LSTM(256, return_sequences=True))

    model.add(Flatten())
    model.add(Dense(512, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(256, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(32, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.001)))

    # opt = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    opt = tfa.optimizers.RectifiedAdam(learning_rate=0.00009)
    opt = tfa.optimizers.SWA(opt)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = recall

    return accuracy, precision, recall, f1, sensitivity, specificity, kappa

def federated_learning(data_path):
    federated_data = FederatedData(data_path, num_clients=3, scaler_type='MinMax')
    federated_data.create_partitions()

    # Get the input shape from the data
    input_shape = federated_data.X.shape[1:]
    global_model = build_sequential_model(input_shape)

    num_clients = 3
    local_epochs = 5
    global_optimizer = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    # Initialize m and v for Adam optimizer
    m = [np.zeros_like(w) for w in global_model.get_weights()]
    v = [np.zeros_like(w) for w in global_model.get_weights()]
    beta1 = 0.9
    beta2 = 0.999
    epsilon = 1e-7
    t = 0

    client_data = []
    for client_idx in range(num_clients):
        x_train, x_val, y_train, y_val, x_test, y_test = federated_data.get_training_and_validation_data(client_idx)
        client_data.append((x_train, x_val, y_train, y_val, x_test, y_test))

    metrics_list = []

    for epoch in range(local_epochs):
        client_models = []

        for client in range(num_clients):
            x_train, x_val, y_train, y_val, x_test, y_test = client_data[client]
            client_model = build_sequential_model(input_shape)
            client_model.set_weights(global_model.get_weights())  # Initialize with global weights
            opt = Adam(learning_rate=0.000009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
            client_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

            # Set up callbacks
            run_name = f"epoch_{epoch}_client_{client}"
            checkpoint_filepath = f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time+Frequency/Alpha/CNN/best_model_{run_name}.h5'
            checkpoint_callback = ModelCheckpoint(
                checkpoint_filepath,
                monitor='val_accuracy',
                save_best_only=True,
                mode='max'
            )
            csv_logger = CSVLogger(f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time+Frequency/Alpha/CNN/training_log_{run_name}.csv', append=True, separator=';')

            history = client_model.fit(x_train, y_train, epochs=100, batch_size=128, validation_data=(x_val, y_val), callbacks=[checkpoint_callback, csv_logger])
            client_models.append(client_model)

            print(history.history)

            # Load the best model before evaluation
            best_model = load_model(checkpoint_filepath)

            # Evaluate client model
            y_pred = (best_model.predict(x_test) > 0.5).astype("int32")
            accuracy, precision, recall, f1, sensitivity, specificity, kappa = compute_metrics(y_test, y_pred)
            metrics_list.append({
                'Client': client,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1': f1,
                'Sensitivity': sensitivity,
                'Specificity': specificity,
                'Kappa': kappa
            })

        global_weights = global_model.get_weights()
        layer_start_idx = 0
        for layer in global_model.layers:
            layer_weights = []
            layer_biases = []
            num_params = len(layer.get_weights())
            for i in range(num_clients):
                layer_params = client_models[i].get_weights()[layer_start_idx:layer_start_idx + num_params]
                if num_params > 0:
                    layer_weights.append(layer_params[0])
                if num_params > 1:
                    layer_biases.append(layer_params[1])

            if len(layer_weights) > 0:
                averaged_layer_weights = np.mean(layer_weights, axis=0)
                if averaged_layer_weights.shape == global_weights[layer_start_idx].shape:
                    global_weights[layer_start_idx] = averaged_layer_weights
                else:
                    print(f"Warning: Shape mismatch for weights at layer {layer.name}, expected {global_weights[layer_start_idx].shape} but got {averaged_layer_weights.shape}")
            if len(layer_biases) > 0:
                averaged_layer_biases = np.mean(layer_biases, axis=0)
                if averaged_layer_biases.shape == global_weights[layer_start_idx + 1].shape:
                    global_weights[layer_start_idx + 1] = averaged_layer_biases
                else:
                    print(f"Warning: Shape mismatch for biases at layer {layer.name}, expected {global_weights[layer_start_idx + 1].shape} but got {averaged_layer_biases.shape}")

            layer_start_idx += num_params

        # Apply FedOpt (Adam) update to global weights
        t += 1
        for i in range(len(global_weights)):
            g_t = global_weights[i] - global_model.get_weights()[i]  # Gradient
            m[i] = beta1 * m[i] + (1 - beta1) * g_t
            v[i] = beta2 * v[i] + (1 - beta2) * (g_t ** 2)
            m_hat = m[i] / (1 - beta1 ** t)
            v_hat = v[i] / (1 - beta2 ** t)
            global_weights[i] -= global_optimizer.learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)

        global_model.set_weights(global_weights)

    metrics_df = pd.DataFrame(metrics_list)
    return global_model, metrics_df

# Example usage
# data_path = 'path/to/your/data.npz'
# global_model, metrics_df = federated_learning(data_path)
# print(metrics_df)


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
global_model_CNN, metrics_df_CNN = federated_learning(Theta_data)

Train shape: (4640, 37, 29), Test shape: (1194, 37, 29)
Train shape: (4418, 37, 29), Test shape: (1416, 37, 29)
Train shape: (4838, 37, 29), Test shape: (996, 37, 29)
Train shape: (4640, 37, 29), Test shape: (1194, 37, 29)
Train shape: (4418, 37, 29), Test shape: (1416, 37, 29)
Train shape: (4838, 37, 29), Test shape: (996, 37, 29)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 37, 256)           22528     
                                                                 
 batch_normalization (Batch  (None, 37, 256)           1024      
 Normalization)                                                  
                                                                 
 max_pooling1d (MaxPooling1  (None, 19, 256)           0         
 D)                                                              
                                                    

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


29/29 [==============================] - 0s 12ms/step - loss: 1.9464 - accuracy: 0.5210 - val_loss: 1.9383 - val_accuracy: 0.5108
Epoch 3/100
29/29 [==============================] - 1s 19ms/step - loss: 1.9291 - accuracy: 0.5207 - val_loss: 1.9221 - val_accuracy: 0.5539
Epoch 4/100
29/29 [==============================] - 0s 13ms/step - loss: 1.9124 - accuracy: 0.5415 - val_loss: 1.9063 - val_accuracy: 0.4838
Epoch 5/100
29/29 [==============================] - 0s 13ms/step - loss: 1.8957 - accuracy: 0.5471 - val_loss: 1.8905 - val_accuracy: 0.5377
Epoch 6/100
29/29 [==============================] - 0s 11ms/step - loss: 1.8801 - accuracy: 0.5361 - val_loss: 1.8751 - val_accuracy: 0.5259
Epoch 7/100
29/29 [==============================] - 0s 12ms/step - loss: 1.8635 - accuracy: 0.5577 - val_loss: 1.8600 - val_accuracy: 0.5496
Epoch 8/100
29/29 [==============================] - 0s 13ms/step - loss: 1.8478 - accuracy: 0.5625 - val_loss: 1.8453 - val_accuracy: 0.4946
Epoch 9/100
29/29 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 1s 23ms/step - loss: 1.9467 - accuracy: 0.5294 - val_loss: 1.9387 - val_accuracy: 0.5385
Epoch 3/100
28/28 [==============================] - 1s 21ms/step - loss: 1.9297 - accuracy: 0.5461 - val_loss: 1.9226 - val_accuracy: 0.5124
Epoch 4/100
28/28 [==============================] - 1s 20ms/step - loss: 1.9141 - accuracy: 0.5068 - val_loss: 1.9067 - val_accuracy: 0.5136
Epoch 5/100
28/28 [==============================] - 0s 13ms/step - loss: 1.8969 - accuracy: 0.5501 - val_loss: 1.8912 - val_accuracy: 0.5226
Epoch 6/100
28/28 [==============================] - 0s 13ms/step - loss: 1.8806 - accuracy: 0.5656 - val_loss: 1.8759 - val_accuracy: 0.5362
Epoch 7/100
28/28 [==============================] - 0s 14ms/step - loss: 1.8649 - accuracy: 0.5617 - val_loss: 1.8609 - val_accuracy: 0.5113
Epoch 8/100
28/28 [==============================] - 1s 20ms/step - loss: 1.8494 - accuracy: 0.5600 - val_loss: 1.8461 - val_accuracy: 0.5441
Epoch 9/100
28/28 

...

29/29 [==============================] - 4s 35ms/step - loss: 0.3865 - accuracy: 0.9690 - val_loss: 0.9788 - val_accuracy: 0.5226
Epoch 2/100
29/29 [==============================] - 1s 20ms/step - loss: 0.3541 - accuracy: 0.9814 - val_loss: 0.9715 - val_accuracy: 0.5388
Epoch 3/100
29/29 [==============================] - 1s 20ms/step - loss: 0.3397 - accuracy: 0.9881 - val_loss: 0.9679 - val_accuracy: 0.5442
Epoch 4/100
29/29 [==============================] - 1s 21ms/step - loss: 0.3280 - accuracy: 0.9938 - val_loss: 0.9608 - val_accuracy: 0.5614
Epoch 5/100
29/29 [==============================] - 1s 21ms/step - loss: 0.3233 - accuracy: 0.9954 - val_loss: 0.9445 - val_accuracy: 0.5894
Epoch 6/100
29/29 [==============================] - 1s 22ms/step - loss: 0.3199 - accuracy: 0.9965 - val_loss: 0.9427 - val_accuracy: 0.5905
Epoch 7/100
29/29 [==============================] - 1s 20ms/step - loss: 0.3212 - accuracy: 0.9965 - val_loss: 0.9110 - val_accuracy: 0.6390
Epoch 8/100
29/29 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/100
28/28 [==============================] - 0s 15ms/step - loss: 0.3542 - accuracy: 0.9822 - val_loss: 0.9624 - val_accuracy: 0.5554
Epoch 3/100
28/28 [==============================] - 0s 15ms/step - loss: 0.3374 - accuracy: 0.9895 - val_loss: 0.9650 - val_accuracy: 0.5554
Epoch 4/100
28/28 [==============================] - 1s 24ms/step - loss: 0.3294 - accuracy: 0.9924 - val_loss: 0.9501 - val_accuracy: 0.5781
Epoch 5/100
28/28 [==============================] - 0s 14ms/step - loss: 0.3254 - accuracy: 0.9943 - val_loss: 0.9522 - val_accuracy: 0.5724
Epoch 6/100
28/28 [==============================] - 1s 21ms/step - loss: 0.3226 - accuracy: 0.9960 - val_loss: 0.9333 - val_accuracy: 0.6029
Epoch 7/100
28/28 [==============================] - 1s 23ms/step - loss: 0.3205 - accuracy: 0.9963 - val_loss: 0.9261 - val_accuracy: 0.6120
Epoch 8/100
28/28 [==============================] - 1s 22ms/step - loss: 0.3179 - accuracy: 0.9972 - val_loss: 0.9033 - val_accuracy: 0.6471
Epoch 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


31/31 [==============================] - 5s 45ms/step - loss: 0.4132 - accuracy: 0.9576 - val_loss: 0.9818 - val_accuracy: 0.5351
Epoch 2/100
31/31 [==============================] - 1s 24ms/step - loss: 0.3553 - accuracy: 0.9811 - val_loss: 0.9784 - val_accuracy: 0.5496
Epoch 3/100
31/31 [==============================] - 0s 15ms/step - loss: 0.3426 - accuracy: 0.9884 - val_loss: 0.9827 - val_accuracy: 0.5413
Epoch 4/100
31/31 [==============================] - 1s 20ms/step - loss: 0.3358 - accuracy: 0.9910 - val_loss: 0.9716 - val_accuracy: 0.5589
Epoch 5/100
31/31 [==============================] - 1s 20ms/step - loss: 0.3313 - accuracy: 0.9935 - val_loss: 0.9523 - val_accuracy: 0.5981
Epoch 6/100
31/31 [==============================] - 0s 14ms/step - loss: 0.3344 - accuracy: 0.9930 - val_loss: 0.9600 - val_accuracy: 0.5816
Epoch 7/100
31/31 [==============================] - 1s 21ms/step - loss: 0.3281 - accuracy: 0.9941 - val_loss: 0.9354 - val_accuracy: 0.6188
Epoch 8/100
31/31 

In [ ]:
metrics_df_CNN.round(3)

,Client,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,Kappa
0,0,0.537,0.540,0.494,0.516,0.494,0.580,0.074
1,1,0.543,0.533,0.691,0.602,0.691,0.395,0.086
2,2,0.570,0.572,0.558,0.565,0.558,0.582,0.141
3,0,0.613,0.626,0.563,0.593,0.563,0.663,0.226
4,1,0.622,0.602,0.723,0.657,0.723,0.521,0.244
5,2,0.662,0.683,0.602,0.640,0.602,0.721,0.323
6,0,0.693,0.692,0.697,0.694,0.697,0.690,0.387
7,1,0.707,0.710,0.701,0.705,0.701,0.713,0.414
8,2,0.740,0.747,0.725,0.736,0.725,0.755,0.480
9,0,0.763,0.750,0.789,0.769,0.789,0.737,0.526


In [ ]:
metrics_df_CNN.round(4).to_csv('/content/drive/MyDrive/EEG Signal /results/FLresults/TF Domain/CNN/Alpha_TF_CNN.csv', index = False)

# GRU


In [8]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, cohen_kappa_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization, LSTM, Dropout, GRU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.backend import clear_session
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
import tensorflow_addons as tfa

class FederatedData:
    def __init__(self, data_path, num_clients, scaler_type='MinMax', test_size=0.2):
        self.data_path = data_path
        self.num_clients = num_clients
        self.scaler_type = scaler_type
        self.test_size = test_size
        self.partitions = []  # Initialize partitions attribute
        self.load_data()
        self.scale_data()
        self.create_partitions()

    def load_data(self):
        try:
            data = np.load(self.data_path, allow_pickle=True)
            self.X = np.moveaxis(data['X'], 1, 2)  # Move axis here
            self.Y = data['Y']
            self.groups = data['Group']  # Assuming 'Group' contains group identifiers
        except KeyError as e:
            raise ValueError(f"Missing expected data field: {e}")
        except FileNotFoundError as e:
            raise ValueError(f"Data file not found: {e}")

    def scale_data(self):
        # Reshape data to 2D array for scaling
        X_reshaped = self.X.reshape(-1, self.X.shape[-1])

        # Select scaler based on input
        if self.scaler_type == 'Standard':
            scaler = StandardScaler()
        elif self.scaler_type == 'MinMax':
            scaler = MinMaxScaler(feature_range=(0, 1))
        else:
            raise ValueError("Unsupported scaler type. Choose either 'Standard' or 'MinMax'.")

        # Fit and transform the data
        scaled_data_reshaped = scaler.fit_transform(X_reshaped)

        # Reshape back to original shape
        self.X = scaled_data_reshaped.reshape(self.X.shape)

    def create_partitions(self):
        gss = GroupShuffleSplit(n_splits=self.num_clients, test_size=self.test_size, random_state=42)
        for train_index, test_index in gss.split(self.X, self.Y, self.groups):
            X_train, X_test = self.X[train_index], self.X[test_index]
            Y_train, Y_test = self.Y[train_index], self.Y[test_index]
            self.partitions.append((X_train, Y_train, X_test, Y_test))
            print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

    def get_training_and_validation_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        partition_X_train, partition_Y_train, partition_X_test, partition_Y_test = self.partitions[client_idx]
        X_train, X_val, Y_train, Y_val = train_test_split(partition_X_train, partition_Y_train, test_size=0.2, random_state=42)
        return X_train, X_val, Y_train, Y_val, partition_X_test, partition_Y_test

    def get_testing_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        _, _, partition_X_test, partition_Y_test = self.partitions[client_idx]
        return partition_X_test, partition_Y_test

def build_sequential_model_gru(input_shape):
    clear_session()
    model = Sequential()

    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same", input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(512, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(filters=1024, kernel_size=3, strides=1, activation='relu'))
    model.add(Dropout(0.5))
    model.add(GRU(256, return_sequences=True))
    model.add(GRU(256, return_sequences=True))

    model.add(Flatten())
    model.add(Dense(512, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(256, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(32, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.001)))

    # opt = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    opt = tfa.optimizers.RectifiedAdam(learning_rate=0.00009)
    opt = tfa.optimizers.SWA(opt)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = recall

    return accuracy, precision, recall, f1, sensitivity, specificity, kappa

def federated_learning_gru(data_path):
    federated_data = FederatedData(data_path, num_clients=3, scaler_type='MinMax')
    federated_data.create_partitions()

    # Get the input shape from the data
    input_shape = federated_data.X.shape[1:]
    global_model = build_sequential_model_gru(input_shape)

    num_clients = 3
    local_epochs = 5
    global_optimizer = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    # Initialize m and v for Adam optimizer
    m = [np.zeros_like(w) for w in global_model.get_weights()]
    v = [np.zeros_like(w) for w in global_model.get_weights()]
    beta1 = 0.9
    beta2 = 0.999
    epsilon = 1e-7
    t = 0

    client_data = []
    for client_idx in range(num_clients):
        x_train, x_val, y_train, y_val, x_test, y_test = federated_data.get_training_and_validation_data(client_idx)
        client_data.append((x_train, x_val, y_train, y_val, x_test, y_test))

    metrics_list = []

    for epoch in range(local_epochs):
        client_models = []

        for client in range(num_clients):
            x_train, x_val, y_train, y_val, x_test, y_test = client_data[client]
            client_model = build_sequential_model_gru(input_shape)
            client_model.set_weights(global_model.get_weights())  # Initialize with global weights
            opt = Adam(learning_rate=0.000009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
            client_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

            # Set up callbacks
            run_name = f"epoch_{epoch}_client_{client}"
            checkpoint_filepath = f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time+Frequency/Alpha/CNN_GRU/best_model_{run_name}.h5'
            checkpoint_callback = ModelCheckpoint(
                checkpoint_filepath,
                monitor='val_accuracy',
                save_best_only=True,
                mode='max'
            )
            csv_logger = CSVLogger(f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time+Frequency/Alpha/CNN_GRU/training_log_{run_name}.csv', append=True, separator=';')

            history = client_model.fit(x_train, y_train, epochs=100, batch_size=128, validation_data=(x_val, y_val), callbacks=[checkpoint_callback, csv_logger])
            client_models.append(client_model)

            print(history.history)

            # Load the best model before evaluation
            best_model = load_model(checkpoint_filepath)

            # Evaluate client model
            y_pred = (best_model.predict(x_test) > 0.5).astype("int32")
            accuracy, precision, recall, f1, sensitivity, specificity, kappa = compute_metrics(y_test, y_pred)
            metrics_list.append({
                'Client': client,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1': f1,
                'Sensitivity': sensitivity,
                'Specificity': specificity,
                'Kappa': kappa
            })

        global_weights = global_model.get_weights()
        layer_start_idx = 0
        for layer in global_model.layers:
            layer_weights = []
            layer_biases = []
            num_params = len(layer.get_weights())
            for i in range(num_clients):
                layer_params = client_models[i].get_weights()[layer_start_idx:layer_start_idx + num_params]
                if num_params > 0:
                    layer_weights.append(layer_params[0])
                if num_params > 1:
                    layer_biases.append(layer_params[1])

            if len(layer_weights) > 0:
                averaged_layer_weights = np.mean(layer_weights, axis=0)
                if averaged_layer_weights.shape == global_weights[layer_start_idx].shape:
                    global_weights[layer_start_idx] = averaged_layer_weights
                else:
                    print(f"Warning: Shape mismatch for weights at layer {layer.name}, expected {global_weights[layer_start_idx].shape} but got {averaged_layer_weights.shape}")
            if len(layer_biases) > 0:
                averaged_layer_biases = np.mean(layer_biases, axis=0)
                if averaged_layer_biases.shape == global_weights[layer_start_idx + 1].shape:
                    global_weights[layer_start_idx + 1] = averaged_layer_biases
                else:
                    print(f"Warning: Shape mismatch for biases at layer {layer.name}, expected {global_weights[layer_start_idx + 1].shape} but got {averaged_layer_biases.shape}")

            layer_start_idx += num_params

        # Apply FedOpt (Adam) update to global weights
        t += 1
        for i in range(len(global_weights)):
            g_t = global_weights[i] - global_model.get_weights()[i]  # Gradient
            m[i] = beta1 * m[i] + (1 - beta1) * g_t
            v[i] = beta2 * v[i] + (1 - beta2) * (g_t ** 2)
            m_hat = m[i] / (1 - beta1 ** t)
            v_hat = v[i] / (1 - beta2 ** t)
            global_weights[i] -= global_optimizer.learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)

        global_model.set_weights(global_weights)

    metrics_df = pd.DataFrame(metrics_list)
    return global_model, metrics_df

# Example usage
# data_path = 'path/to/your/data.npz'
# global_model, metrics_df = federated_learning(data_path)
# print(metrics_df)



/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [9]:
global_model_gru, metrics_df_gru = federated_learning_gru(Theta_data)

Train shape: (4640, 37, 29), Test shape: (1194, 37, 29)
Train shape: (4418, 37, 29), Test shape: (1416, 37, 29)
Train shape: (4838, 37, 29), Test shape: (996, 37, 29)
Train shape: (4640, 37, 29), Test shape: (1194, 37, 29)
Train shape: (4418, 37, 29), Test shape: (1416, 37, 29)
Train shape: (4838, 37, 29), Test shape: (996, 37, 29)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 37, 256)           22528     
                                                                 
 batch_normalization (Batch  (None, 37, 256)           1024      
 Normalization)                                                  
                                                                 
 max_pooling1d (MaxPooling1  (None, 19, 256)           0         
 D)                                                              
                                                    

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


29/29 [==============================] - 33s 235ms/step - loss: 1.7053 - accuracy: 0.4949 - val_loss: 1.7002 - val_accuracy: 0.4871
Epoch 2/100
29/29 [==============================] - 1s 30ms/step - loss: 1.6955 - accuracy: 0.5003 - val_loss: 1.6905 - val_accuracy: 0.4978
Epoch 3/100
29/29 [==============================] - 1s 20ms/step - loss: 1.6859 - accuracy: 0.5046 - val_loss: 1.6810 - val_accuracy: 0.4849
Epoch 4/100
29/29 [==============================] - 1s 20ms/step - loss: 1.6764 - accuracy: 0.5032 - val_loss: 1.6715 - val_accuracy: 0.4849
Epoch 5/100
29/29 [==============================] - 1s 19ms/step - loss: 1.6669 - accuracy: 0.5046 - val_loss: 1.6621 - val_accuracy: 0.4849
Epoch 6/100
29/29 [==============================] - 1s 29ms/step - loss: 1.6573 - accuracy: 0.5078 - val_loss: 1.6528 - val_accuracy: 0.5075
Epoch 7/100
29/29 [==============================] - 1s 34ms/step - loss: 1.6480 - accuracy: 0.5127 - val_loss: 1.6435 - val_accuracy: 0.5119
Epoch 8/100
29/2

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 13s 210ms/step - loss: 1.7054 - accuracy: 0.5028 - val_loss: 1.7005 - val_accuracy: 0.4943
Epoch 2/100
28/28 [==============================] - 1s 25ms/step - loss: 1.6959 - accuracy: 0.5031 - val_loss: 1.6912 - val_accuracy: 0.4943
Epoch 3/100
28/28 [==============================] - 1s 32ms/step - loss: 1.6866 - accuracy: 0.5175 - val_loss: 1.6819 - val_accuracy: 0.4966
Epoch 4/100
28/28 [==============================] - 1s 33ms/step - loss: 1.6774 - accuracy: 0.5113 - val_loss: 1.6728 - val_accuracy: 0.4989
Epoch 5/100
28/28 [==============================] - 1s 33ms/step - loss: 1.6682 - accuracy: 0.5232 - val_loss: 1.6638 - val_accuracy: 0.5011
Epoch 6/100
28/28 [==============================] - 1s 21ms/step - loss: 1.6592 - accuracy: 0.5136 - val_loss: 1.6548 - val_accuracy: 0.4966
Epoch 7/100
28/28 [==============================] - 1s 42ms/step - loss: 1.6500 - accuracy: 0.5272 - val_loss: 1.6459 - val_accuracy: 0.5023
Epoch 8/100
28/2

...

29/29 [==============================] - 13s 232ms/step - loss: 0.5031 - accuracy: 0.9057 - val_loss: 0.9361 - val_accuracy: 0.5172
Epoch 2/100
29/29 [==============================] - 1s 32ms/step - loss: 0.3680 - accuracy: 0.9534 - val_loss: 0.9305 - val_accuracy: 0.5216
Epoch 3/100
29/29 [==============================] - 1s 20ms/step - loss: 0.3441 - accuracy: 0.9612 - val_loss: 0.9311 - val_accuracy: 0.5205
Epoch 4/100
29/29 [==============================] - 1s 32ms/step - loss: 0.3384 - accuracy: 0.9609 - val_loss: 0.9278 - val_accuracy: 0.5259
Epoch 5/100
29/29 [==============================] - 1s 30ms/step - loss: 0.3398 - accuracy: 0.9604 - val_loss: 0.9101 - val_accuracy: 0.5485
Epoch 6/100
29/29 [==============================] - 1s 33ms/step - loss: 0.3325 - accuracy: 0.9626 - val_loss: 0.9048 - val_accuracy: 0.5528
Epoch 7/100
29/29 [==============================] - 1s 29ms/step - loss: 0.3242 - accuracy: 0.9661 - val_loss: 0.8991 - val_accuracy: 0.5603
Epoch 8/100
29/2

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 12s 213ms/step - loss: 0.4226 - accuracy: 0.9335 - val_loss: 0.9380 - val_accuracy: 0.5068
Epoch 2/100
28/28 [==============================] - 1s 30ms/step - loss: 0.3545 - accuracy: 0.9539 - val_loss: 0.9319 - val_accuracy: 0.5113
Epoch 3/100
28/28 [==============================] - 1s 25ms/step - loss: 0.3396 - accuracy: 0.9612 - val_loss: 0.9321 - val_accuracy: 0.5136
Epoch 4/100
28/28 [==============================] - 0s 17ms/step - loss: 0.3296 - accuracy: 0.9626 - val_loss: 0.9364 - val_accuracy: 0.5102
Epoch 5/100
28/28 [==============================] - 1s 25ms/step - loss: 0.3283 - accuracy: 0.9643 - val_loss: 0.9233 - val_accuracy: 0.5158
Epoch 6/100
28/28 [==============================] - 1s 26ms/step - loss: 0.3213 - accuracy: 0.9700 - val_loss: 0.9150 - val_accuracy: 0.5238
Epoch 7/100
28/28 [==============================] - 1s 28ms/step - loss: 0.3090 - accuracy: 0.9734 - val_loss: 0.8964 - val_accuracy: 0.5645
Epoch 8/100
28/2

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


31/31 [==============================] - 13s 239ms/step - loss: 0.4841 - accuracy: 0.9116 - val_loss: 0.9313 - val_accuracy: 0.5186
Epoch 2/100
31/31 [==============================] - 1s 28ms/step - loss: 0.4098 - accuracy: 0.9328 - val_loss: 0.9276 - val_accuracy: 0.5227
Epoch 3/100
31/31 [==============================] - 1s 17ms/step - loss: 0.3921 - accuracy: 0.9424 - val_loss: 0.9303 - val_accuracy: 0.5227
Epoch 4/100
31/31 [==============================] - 1s 25ms/step - loss: 0.3561 - accuracy: 0.9530 - val_loss: 0.9213 - val_accuracy: 0.5279
Epoch 5/100
31/31 [==============================] - 1s 17ms/step - loss: 0.3690 - accuracy: 0.9473 - val_loss: 0.9313 - val_accuracy: 0.5196
Epoch 6/100
31/31 [==============================] - 1s 24ms/step - loss: 0.3509 - accuracy: 0.9579 - val_loss: 0.9130 - val_accuracy: 0.5424
Epoch 7/100
31/31 [==============================] - 1s 26ms/step - loss: 0.3562 - accuracy: 0.9571 - val_loss: 0.9141 - val_accuracy: 0.5444
Epoch 8/100
31/3

In [10]:
metrics_df_gru

,Client,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,Kappa
0,0,0.533501,0.541841,0.433836,0.481860,0.433836,0.633166,0.067002
1,1,0.558616,0.550920,0.634181,0.589626,0.634181,0.483051,0.117232
2,2,0.590361,0.629310,0.439759,0.517730,0.439759,0.740964,0.180723
3,0,0.601340,0.586305,0.688442,0.633282,0.688442,0.514238,0.202680
4,1,0.612288,0.650284,0.485876,0.556184,0.485876,0.738701,0.224576
5,2,0.656627,0.669565,0.618474,0.643006,0.618474,0.694779,0.313253
6,0,0.690117,0.671192,0.745394,0.706349,0.745394,0.634841,0.380235
7,1,0.691384,0.727731,0.611582,0.664620,0.611582,0.771186,0.382768
8,2,0.735944,0.753780,0.700803,0.726327,0.700803,0.771084,0.471888
9,0,0.732831,0.768340,0.666667,0.713901,0.666667,0.798995,0.465662


In [11]:
metrics_df_gru.round(4).to_csv('/content/drive/MyDrive/EEG Signal /results/FLresults/TF Domain/GRU/Alpha_tf_GRU.csv', index = False)